In [1]:
%matplotlib inline

from __future__ import print_function, division

import numpy as np
import math

import matplotlib

from matplotlib import pyplot as plt
import matplotlib.colors as colors
from matplotlib import rcParams, rc
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
from matplotlib import gridspec, ticker

import sys
import xpsi
                   
import os

from xpsi import PostProcessing
PostProcessing.publication_rc_settings()
PostProcessing.random_seed = 0

from xpsi.global_imports import _dpr, _keV, _k_B

/=============================================\
| X-PSI: X-ray Pulse Simulation and Inference |
|---------------------------------------------|
|              Version: 0.1                 |
\=============================================/

Imported GetDist version: 0.3.1
Imported nestcheck version: 0.2.0


In [2]:
PostProcessing.publication_rc_settings()

In [3]:
from CustomData import CustomData
from CustomInstrument import CustomInstrument
from CustomInterstellar import CustomInterstellar
from CustomPulse import CustomPulse
from CustomSpacetime import CustomSpacetime
from CustomPrior import CustomPrior
from CustomPhotosphere import CustomPhotosphere

data = CustomData.from_SWG('../../j0030/data/NICER_J0030_PaulRay_fixed_evt_25to299__preprocessed.txt', 1936864.0)

NICER = CustomInstrument.from_SWG(num_params=3,
                    bounds=[(0.5,1.5),(0.0,1.0),(0.5,1.5)],
                    ARF = '../../j0030/model_data/ni_xrcall_onaxis_v1.02_arf.txt',
                    RMF = '../../j0030/model_data/nicer_upd_d49_matrix.txt',
                    ratio = '../../j0030/model_data/crab_ratio_SA80_d49.txt',
                    max_input=700,
                    min_input=0,
                    chan_edges = '../../j0030/model_data/nicer_upd_energy_bounds.txt')

interstellar = CustomInterstellar.from_SWG('../../j0030/model_data/interstellar_phot_frac.txt',
                                           num_params = 1,
                                           bounds = [(0.0, 5.0)])

pulse = CustomPulse(tag = 'all',
                    num_params = 2,
                    bounds = [(0.35, 0.55), (-0.25,0.75)],
                    data = data,
                    instrument = NICER,
                    interstellar = interstellar,
                    energies_per_interval = 0.25,
                    fast_rel_energies_per_interval = 0.5,
                    default_energy_spacing = 'logspace',
                    adaptive_energies = False,
                    adapt_exponent = None,
                    store = False,
                    workspace_intervals = 1000,
                    epsrel = 1.0e-8,
                    epsilon = 1.0e-3,
                    sigmas = 10.0)

from xpsi.global_imports import _c, _G, _M_s, _dpr, gravradius

bounds = [(0.235, 0.415),
          (1.0, 3.0),
          (3.0 * gravradius(1.0), 16.0),
          (0.001, math.pi/2.0)]

spacetime = CustomSpacetime(num_params = 4, bounds = bounds, S = 1.0/(4.87e-3))

bounds = [(0.001, math.pi - 0.001),
          (0.001, math.pi/2.0 - 0.001),
          (5.1, 6.8)]

primary = xpsi.Spot(num_params=3, bounds=bounds,
                    symmetry=True,
                    hole=False,
                    cede=False,
                    concentric=False,
                    sqrt_num_cells=24,
                    min_sqrt_num_cells=10,
                    max_sqrt_num_cells=64,
                    do_fast=False,
                    fast_sqrt_num_cells=8,
                    fast_min_sqrt_num_cells=8,
                    fast_max_sqrt_num_cells=16,
                    fast_num_leaves=32,
                    fast_num_rays=100,
                    num_leaves=80,
                    num_rays=200)

bounds = [(0.001, math.pi - 0.001),
          (0.001, math.pi/2.0 - 0.001),
          (0.001, math.pi - 0.001),
          (0.0, 2.0),
          (0.0, 2.0*math.pi),
          (5.1, 6.8)]

secondary = xpsi.Spot(num_params=6, bounds=bounds,
                      symmetry=True,
                      hole=True,
                      cede=False,
                      concentric=False,
                      sqrt_num_cells=24,
                      min_sqrt_num_cells=10,
                      max_sqrt_num_cells=64,
                      do_fast=False,
                      fast_sqrt_num_cells=8,
                      fast_min_sqrt_num_cells=8,
                      fast_max_sqrt_num_cells=16,
                      fast_num_leaves=32,
                      fast_num_rays=100,
                      num_leaves=80,
                      num_rays=200,
                      is_secondary=True)

from xpsi import TwoSpots

spot = TwoSpots((primary, secondary))

photosphere = CustomPhotosphere(num_params = 0, bounds = [],
                                tag = 'all', spot = spot, elsewhere = None)

photosphere.spot_atmosphere = '../../j0030/model_data/nsx_H_v171019.out'

star = xpsi.Star(spacetime = spacetime, photospheres = photosphere)

likelihood = xpsi.Likelihood(star = star, pulses = pulse, threads=1)

prior = CustomPrior(bounds=likelihood.bounds, spacetime=spacetime)

likelihood.prior = prior

In [4]:
np.sum(pulse._precomp) # should be: -24413582.28500478

-7173325.85719545

In [5]:
prior.estimate_hypercube_frac(4)

>> Estimating fractional hypervolume of the unit hypercube with finite prior density:
Requiring 1E+04 draws from the prior support for Monte Carlo estimation.
The support of the joint prior occupies an estimated fraction 0.3629 of the hypervolume within the unit hypercube.
>> Fractional hypervolume estimated.


0.36293688527565054

In [6]:
prior._bounds

[(0.235, 0.415),
 (1.0, 3.0),
 (4.429875115500001, 16.0),
 (0.001, 1.5707963267948966),
 (0.001, 3.1405926535897932),
 (0.001, 1.5697963267948967),
 (5.1, 6.8),
 (0.001, 3.1405926535897932),
 (0.001, 1.5697963267948967),
 (0.001, 3.1405926535897932),
 (0.0, 2.0),
 (0.0, 6.283185307179586),
 (5.1, 6.8),
 (0.0, 5.0),
 (0.5, 1.5),
 (0.0, 1.0),
 (0.5, 1.5),
 (0.35, 0.55),
 (-0.25, 0.75)]

In [7]:
prior.inverse_sample_and_transform(np.random.rand(19))

[0.33433927805858343,
 2.914570367764229,
 7.093923352465554,
 0.9235098094785921,
 2.1917142176998987,
 1.0513726807038684,
 6.145764318369396,
 1.6542146907153328,
 0.7710795281438787,
 2.4013422814492897,
 1.2787614027734422,
 -0.759624134174976,
 5.64762936140785,
 4.369791079261326,
 1.0399406542928644,
 0.1805418047251146,
 0.9846387039275736,
 -0.4651485858881115,
 0.11330283559167087,
 0.6066780493216095,
 -0.5076818746295635,
 -0.8866971644083291,
 1.3970106335149604,
 1.2787614027734422,
 0.3126134151404561,
 0.6934329153491041]

In [8]:
print(CustomPrior.__doc__)

 A custom (joint) prior distribution.

    Currently tailored to the NICER light-curve SWG model specification.

    Source: PSR J0030+0451
    Model variant: ST+PST

    Parameter vector:

    * p[0] = distance (kpc)
    * p[1] = (rotationally deformed) gravitational mass (solar masses)
    * p[2] = coordinate equatorial radius (km)
    * p[3] = inclination of Earth to rotational axis (radians)
    * p[4] = primary centre colatitude (radians)
    * p[5] = primary angular radius (radians)
    * p[6] = primary log10(comoving NSX FIH effective temperature [K])
    * p[7] = secondary centre colatitude (radians)
    * p[8] = secondary angular radius (radians)
    * p[9] = secondary hole colatitude (radians)
    * p[10] = secondary hole angular radius (radians)
    * p[11] = secondary hole azimuth (radians); periodic
    * p[12] = secondary log10(comoving NSX FIH effective temperature [K])
    * p[13] = hydrogen column density (10^20 cm^-2)
    * p[14] = instrument parameter a
    * p[15] =

In [9]:
# names of free parameters ordered as in sample files
names = ['distance', 'mass', 'radius', 'cos_inclination',
         'p__super_colatitude', 'p__super_radius', 'p__super_temperature',
         's__super_colatitude', 's__super_radius', 's__omit_colatitude',
         's__omit_radius', 's__omit_azimuth', 's__super_temperature',
         'column_density',
         'alpha', 'beta', 'gamma',
         'p__phase_shift', 's__phase_shift']

# names of derived variables of interest
names += ['compactness',
          's__annulus_width',
          's__transformed_phase',
          's__f',
          's__xi',
          's__super_offset_fraction',
          's__super_offset_azi']

bounds = {'distance': (0.235, 0.415),
                  'mass': (1.0, 3.0),
                  'radius': (3.0 * gravradius(1.0), 16.0),
                  'cos_inclination': (0.0, math.cos(0.001)),
                  'p__super_colatitude': (0.001, math.pi - 0.001),
                  'p__super_radius': (0.001, math.pi/2.0 - 0.001),
                  'p__super_temperature': (5.1, 6.8),
                  's__super_colatitude': (0.001, math.pi - 0.001),
                  's__super_radius': (0.001, math.pi/2.0 - 0.001),
                  's__super_temperature': (5.1, 6.8),
                  's__omit_colatitude': (0.001, math.pi - 0.001),
                  's__omit_radius': (0.0, math.pi/2.0 - 0.001),
                  's__omit_azimuth': (-math.pi, math.pi),
                  'column_density': (0.0, 5.0),
                  'alpha': (0.5,1.5),
                  'beta': (0.0,1.0),
                  'gamma': (0.5,1.5),
                  'p__phase_shift': (-0.5,0.5),
                  's__phase_shift': (-0.5,0.5),
                  'compactness': (gravradius(1.0)/16.0, 1.0/3.0),
                  's__annulus_width': (-math.pi/2.0 + 0.001, math.pi/2.0 - 0.001),
                  's__transformed_phase': (-1.0, 0.0),
                  's__f': (0.0, 2.0),
                  's__xi': (0.001, math.pi/2.0 - 0.001),
                  's__super_offset_fraction': (0.0, 1.0),
                  's__super_offset_azi': (0.0, 2.0)}

labels = {'distance': r"D\;\mathrm{[kpc]}",
                  'mass': r"M\;\mathrm{[M}_{\odot}\mathrm{]}",
                  'radius': r"R_{\mathrm{eq}}\;\mathrm{[km]}",
                  'cos_inclination': r"\cos(i)",
                  'p__super_colatitude': r"\Theta_{p}\;\mathrm{[rad]}",
                  'p__super_radius': r"\zeta_{p}\;\mathrm{[rad]}",
                  'p__super_temperature': r"\mathrm{log10}(T_{p}\;[\mathrm{K}])",
                  's__super_colatitude': r"\Theta_{s}\;\mathrm{[rad]}",
                  's__super_radius': r"\zeta_{s}\;\mathrm{[rad]}",
                  's__super_temperature': r"\mathrm{log10}(T_{s}\;[\mathrm{K}])",
                  's__omit_colatitude': r"\Theta_{s}\;\mathrm{[rad]}",
                  's__omit_radius': r"\psi_{s}\;\mathrm{[rad]}",
                  's__omit_azimuth': r".", # won't bother plotting so no need for symbol
                  'column_density': r"N_{\mathrm{H}}\;\mathrm{[10^{20}\;cm^{-2}]}",
                  'alpha': r"\alpha",
                  'beta': r"\beta",
                  'gamma': r"\gamma",
                  'p__phase_shift': r"\phi_{p}\;\mathrm{[cycles]}",
                  's__phase_shift': r"\phi_{s}\;\mathrm{[cycles]}",
                  'compactness': r"M/R_{\mathrm{eq}}",
                  's__annulus_width': r"\psi^{+}\;\mathrm{[rad]}",
                  's__transformed_phase': r"\phi_{s}\;\mathrm{[cycles]}",
                  's__f': r"f_{s}",
                  's__xi': r"\xi_{s}\;\mathrm{[rad]}",
                  's__super_offset_fraction': r"\kappa_{s}",
                  's__super_offset_azi': r"\varphi_{s}\;\mathrm{[\pi\,radians]}"} 

In [10]:
getdist_kde_settings = {'ignore_rows': 0,
                         'min_weight_ratio': 1.0e-10,
                         'contours': [0.683, 0.954, 0.997],
                         'credible_interval_threshold': 0.001,
                         'range_ND_contour': 0,
                         'range_confidence': 0.001,
                         'fine_bins': 1024,
                         'smooth_scale_1D': 0.4,
                         'num_bins': 100,
                         'boundary_correction_order': 1,
                         'mult_bias_correction_order': 1,
                         'smooth_scale_2D': 0.4,
                         'max_corr_2D': 0.99,
                         'fine_bins_2D': 512,
                         'num_bins_2D': 40}

In [11]:
!ls /home/xpsi/A_NICER_VIEW_OF_PSR_J0030p0451/ST_PST/run1/

!mv run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformeddead-birth.txt run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformed-dead-birth.txt
# !mv run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformedphys_live-birth.txt run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformedphys_live-birth.txt

'combined_IDs_run 1_.stats'
'combined_IDs_run 1_.txt'
'combined_IDs_run 1__dead-birth.txt'
'combined_IDs_run 1__dead.txt'
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformed-dead-birth.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformed-phys_live-birth.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformed.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformeddead-birth.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1_transformedphys_live-birth.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1dead-birth.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1ev.dat
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1live.points
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1phys_live-birth.txt
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1phys_live.points
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1post_equal_weights.dat
 run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1resume.dat
 run1_nlive1000_eff0.3_noCONST_no

In [12]:
pp = PostProcessing.PostProcessor.load_runs(['/home/xpsi/A_NICER_VIEW_OF_PSR_J0030p0451/ST_PST/run1/run1_nlive1000_eff0.3_noCONST_noMM_noIS_tol-1'],
                                            ['run 1'],
                                            use_nestcheck=[True],
                                            base_dir='/home/xpsi/A_NICER_VIEW_OF_PSR_J0030p0451/ST_PST/run1/',
                                            kde_settings=getdist_kde_settings,
                                            names=names,
                                            bounds=bounds,
                                            labels=labels,
                                            implementation='multinest',
                                            transform=prior.transform,
                                            likelihood=likelihood)

/opt/conda/envs/xpsi/lib/python2.7/site-packages/getdist/chains.py:1149: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  weights = None if self.chains[0].weights is None else np.hstack((chain.weights for chain in self.chains))
/opt/conda/envs/xpsi/lib/python2.7/site-packages/getdist/chains.py:1150: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  loglikes = None if self.chains[0].loglikes is None else np.hstack((chain.loglikes for chain in self.chains))
/opt/conda/envs/xpsi/lib/python2.7/site-packages/getdist/chains.py:1151: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as g

In [ ]:
_ = pp.plot_posteriorDensity(
     params=['radius','compactness','mass'],
     run_IDs=OrderedDict([('ST-U', ['mode separation', 'run 2', 'run 1',]),]),
     prior_density=True,
     KL_divergence=True,
     ndraws=5e4,
     combine=True, combine_all=True, only_combined=False, overwrite_combined=True,
     param_plot_lims={'radius': (7.5,16.0), 'compactness': (0.11,0.20), 'mass': (1.0,2.0)},
     bootstrap_estimators=False,
     bootstrap_density=False,
     n_simulate=200,
     crosshairs=False,
     write=False,
     ext='.png',
     maxdots=3000,
     root_filename='STU_spacetime',
     credible_interval_1d=True,
     annotate_credible_interval=True,
     compute_all_intervals=False,
     sixtyeight=True,
     x_label_rotation=45.0,
     num_plot_contours=3,
     subplot_size=4.0,
     legend_corner_coords=(0.675,0.8),
     legend_frameon=False,
     scale_attrs=OrderedDict([('legend_fontsize', 2.0),
                              ('lab_fontsize', 1.35),
                              ('axes_fontsize', 'lab_fontsize'),
                             ]
                            ),
     colormap='Reds',
     shaded=True,
     shade_root_index=-1,
     rasterized_shade=True,
     no_ylabel=True,
     no_ytick=True,
     lw=1.0,
     lw_1d=1.0,
     filled=False,
     normalize=True,
     veneer=True,
     tqdm_kwargs={'disable': False},
     lengthen=2.0,
     embolden=1.0,
     nx=500,
     scale_ymax=1.1)

In [13]:
pp.plot_posteriorDensity(params=['distance', 'mass', 'radius', 'cos_inclination'],
                         run_IDs=['run 1'],
                         prior_density=True,
                         KL_divergence=True,
                         ndraws=1e5,
                         combine=False, combine_all=False, only_combined=False,
                         param_plot_lims={},
                         bootstrap_estimators=True,
                         bootstrap_density=False,
                         crosshairs=True,
                         write=True,
                         ext='.png',
                         maxdots=4000,
                         root_filename='twospots',
                         credible_interval_1d=True,
                         annotate_credible_interval=True,
                         compute_all_intervals=True,
                         sixtyeight=True,
                         x_label_rotation=45.0,
                         num_plot_contours=3,
                         subplot_size=4.0,
                         tick_prune=None,
                         legend_loc='lower left',
                         legend_corner_coords=(0.65,0.7),
                         legend_frameon=False,
                         scale_attrs={'legend_fontsize': 5.0,
                                      'axes_fontsize': 'lab_fontsize',
                                      'lab_fontsize': 2.0},
                         colormap='Reds',
                         shaded=True,
                         shade_root_index=-1,
                         rasterized_shade=True,
                         no_ylabel=True,
                         no_ytick=True,
                         lw=1.0,
                         lw_1d=1.0,
                         filled=False,
                         normalize=True,
                         veneer=True,
                         tqdm_kwargs={'disable': False},
                         lengthen=2.0,
                         embolden=1.0,
                         nx=500,
                         scale_ymax=1.1,
                         n_simulate=200)

>> Executing posterior density estimation...
>> Curating set of runs for posterior plotting...
>> Run set curated.
>> Constructing lower-triangle posterior density plot via Gaussian KDE:


TypeError: plot_1d() got multiple values for keyword argument 'no_ytick'

Error in callback <function post_execute at 0x7faefb91ba28> (for post_execute):


/opt/conda/envs/xpsi/lib/python2.7/site-packages/matplotlib/font_manager.py:1331: UserWarning: findfont: Font family [u'serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


OSError: [Errno 2] No such file or directory: 'latex'

OSError: [Errno 2] No such file or directory: 'latex'

<Figure size 1152x1152 with 1 Axes>

In [ ]:
!pip install --user latex

In [14]:
import subprocess
subprocess.check_call(["latex"])

OSError: [Errno 2] No such file or directory

In [15]:
!whereis latex

latex:
